In [4]:
from point_coordinates_regression import *
from Deformation_gradient import *
from math import acos,sqrt,pi

%matplotlib tk

In [5]:
# trying to include a distance quality to a point by smapling points around a specific point

polygons_initial=load_dataset('12_polygons.pkl')
point_coordinates_initial=load_dataset('12_point_coordinates_del.pkl')
number_of_insertion_points=load_dataset('12_nb_of_points_del.pkl')

In [7]:
nb_of_points=12
polygons_reshaped,point_coordinates=reshape_data(polygons_initial,point_coordinates_initial,number_of_insertion_points,nb_of_points)

In [102]:
contour=np.delete(polygons_reshaped[0],24).reshape(12,2)
optimal_point=point_coordinates[0][0]

In [136]:
plot_contour(contour)
plt.scatter(optimal_point[0],optimal_point[1],marker='x')

In [622]:

random_points=[]
points_qualities=[]
training_inputs=[]
training_outputs=[]
for i in np.linspace(0,1,10): #number of radius steps
    points_quality=1-i
    radius=i
    for j in range(1,20): # number of point sampling on circle
        angle = 1/2*j+i+pi
        random_point = np.array([radius * np.cos(angle)+optimal_point[0],radius * np.sin(angle)+optimal_point[1]])
        random_points.append(random_point)
        points_qualities.append(points_quality)
        training_inputs.append(np.hstack([contour.reshape(24),points_quality]))
        training_outputs.append(random_point)
random_points=np.array( random_points)
training_inputs=np.array(training_inputs)
training_outputs=np.array(training_outputs)

points_qualities=points_qualities

In [657]:
plt.scatter(random_points[:,0],random_points[:,1],marker='d')

In [624]:
x_tensor=torch.from_numpy(training_inputs).type(torch.FloatTensor)
y_tensor=torch.from_numpy(training_outputs.reshape(len(training_outputs),1,2)).type(torch.FloatTensor)
x_variable=Variable(x_tensor)
y_variable=Variable(y_tensor)

In [625]:
my_net=Net(x_variable.size()[1],y_variable.size()[2],nb_of_hidden_layers=1, nb_of_hidden_nodes=15,batch_normalization=True)
torch.cuda.empty_cache()


In [626]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4,weight_decay=0.3)
loss_func =torch.nn.MSELoss(size_average=False) 


In [627]:
if  torch.cuda.is_available():
    loss_func.cuda()
        
    x_variable , y_variable=x_variable.cuda(), y_variable.cuda()

    print("cuda activated")

cuda activated


In [628]:
batch_size=int(x_variable.size()[0] )
nb_of_epochs=23000
my_net.cuda()
#my_net.cpu()

# Train the network #
my_net.train()
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss= loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        sum_loss+=loss.data[0]
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients

        optimizer.step()        # apply gradients
    if t%10==0: 

        print("Epoch:",t,"Training Loss:",sum_loss/(x_variable.size(0)))
        my_net.train()


Epoch: 0 Training Loss: 0.5299932379471628
Epoch: 10 Training Loss: 0.5245864868164063
Epoch: 20 Training Loss: 0.5193093550832648
Epoch: 30 Training Loss: 0.5141550164473684
Epoch: 40 Training Loss: 0.5091319033974095
Epoch: 50 Training Loss: 0.5042471233167146
Epoch: 60 Training Loss: 0.4995009572882401
Epoch: 70 Training Loss: 0.49487617894222863
Epoch: 80 Training Loss: 0.4903793736508018
Epoch: 90 Training Loss: 0.48599917763157896
Epoch: 100 Training Loss: 0.4817303306178043
Epoch: 110 Training Loss: 0.47756580553556743
Epoch: 120 Training Loss: 0.4734987359297903
Epoch: 130 Training Loss: 0.4695269132915296
Epoch: 140 Training Loss: 0.46564700478001647
Epoch: 150 Training Loss: 0.46185632002981086
Epoch: 160 Training Loss: 0.4581630506013569
Epoch: 170 Training Loss: 0.4545551099275288
Epoch: 180 Training Loss: 0.4510395250822368
Epoch: 190 Training Loss: 0.44760509290193257
Epoch: 200 Training Loss: 0.4442664297003495
Epoch: 210 Training Loss: 0.441008156224301
Epoch: 220 Train

Epoch: 1820 Training Loss: 0.34591068468595804
Epoch: 1830 Training Loss: 0.34579174644068666
Epoch: 1840 Training Loss: 0.3456723263389186
Epoch: 1850 Training Loss: 0.34555298654656663
Epoch: 1860 Training Loss: 0.3454332853618421
Epoch: 1870 Training Loss: 0.34531374479594984
Epoch: 1880 Training Loss: 0.34519380268297695
Epoch: 1890 Training Loss: 0.34507374010587993
Epoch: 1900 Training Loss: 0.34495355706465874
Epoch: 1910 Training Loss: 0.3448332937140214
Epoch: 1920 Training Loss: 0.3447115044844778
Epoch: 1930 Training Loss: 0.34458762721011515
Epoch: 1940 Training Loss: 0.34446326807925576
Epoch: 1950 Training Loss: 0.34433862786543995
Epoch: 1960 Training Loss: 0.3442134656404194
Epoch: 1970 Training Loss: 0.3440883435701069
Epoch: 1980 Training Loss: 0.34396209716796877
Epoch: 1990 Training Loss: 0.34383275885331005
Epoch: 2000 Training Loss: 0.3437024568256579
Epoch: 2010 Training Loss: 0.34357115093030427
Epoch: 2020 Training Loss: 0.34343928286903785
Epoch: 2030 Training

Epoch: 3610 Training Loss: 0.3360413400750411
Epoch: 3620 Training Loss: 0.33603985435084294
Epoch: 3630 Training Loss: 0.33603840878135277
Epoch: 3640 Training Loss: 0.3360370033665707
Epoch: 3650 Training Loss: 0.3360356381064967
Epoch: 3660 Training Loss: 0.33603431300113074
Epoch: 3670 Training Loss: 0.33603302805047286
Epoch: 3680 Training Loss: 0.3360318234092311
Epoch: 3690 Training Loss: 0.3360306187679893
Epoch: 3700 Training Loss: 0.3360294141267475
Epoch: 3710 Training Loss: 0.3360282897949219
Epoch: 3720 Training Loss: 0.33602718554045025
Epoch: 3730 Training Loss: 0.33602604113127055
Epoch: 3740 Training Loss: 0.3360250372635691
Epoch: 3750 Training Loss: 0.33602401331851356
Epoch: 3760 Training Loss: 0.33602296929610403
Epoch: 3770 Training Loss: 0.33602196542840257
Epoch: 3780 Training Loss: 0.33602102179276316
Epoch: 3790 Training Loss: 0.3360200380024157
Epoch: 3800 Training Loss: 0.33601913452148435
Epoch: 3810 Training Loss: 0.33601817080849095
Epoch: 3820 Training L

KeyboardInterrupt: 

In [631]:
for i in np.linspace(0,0.5,20):
    quality_demand=i
    quality_label=str(quality_demand)
    x_test=np.hstack([contour.reshape(24),quality_demand])
    x_test_variable=Variable(torch.from_numpy(x_test).type(torch.FloatTensor))
    x_test_variable=x_test_variable.expand(100,25)

    my_net.eval()
    predicted_point=my_net(x_test_variable.cuda())
    predicted_point=predicted_point[0].data.cpu().numpy()
    plt.scatter(predicted_point[0],predicted_point[1],label=quality_label)
plot_contour(contour)
plt.scatter(optimal_point[0],optimal_point[1],marker='x')
plt.legend()

In [619]:
plt.clf()

In [655]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou
Dependencies:
torch: 0.4
matplotlib
numpy
"""
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

# torch.manual_seed(1)    # reproducible

# Hyper Parameters
TIME_STEP = 10     # rnn time step
INPUT_SIZE = 1      # rnn input size
LR = 0.02           # learning rate

# show data
steps = np.linspace(0, np.pi*2, 100, dtype=np.float32)
x_np = np.sin(steps)    # float32 for converting torch FloatTensor
y_np = np.cos(steps)
plt.plot(steps, y_np, 'r-', label='target (cos)')
plt.plot(steps, x_np, 'b-', label='input (sin)')
plt.legend(loc='best')
plt.show()


class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size=INPUT_SIZE,
            hidden_size=32,     # rnn hidden unit
            num_layers=1,       # number of rnn layer
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(32, 1)

    def forward(self, x, h_state):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        r_out, h_state = self.rnn(x, h_state)

        outs = []    # save all predictions
        for time_step in range(r_out.size(1)):    # calculate output for each time step
            outs.append(self.out(r_out[:, time_step, :]))
        return torch.stack(outs, dim=1), h_state

        # instead, for simplicity, you can replace above codes by follows
        # r_out = r_out.view(-1, 32)
        # outs = self.out(r_out)
        # return outs, h_state

rnn = RNN()
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.MSELoss()

h_state = None      # for initial hidden state

plt.figure(1, figsize=(12, 5))
plt.ion()           # continuously plot

for step in range(100):
    start, end = step * np.pi, (step+1)*np.pi   # time range
    # use sin predicts cos
    steps = np.linspace(start, end, TIME_STEP, dtype=np.float32)
    x_np = np.sin(steps)    # float32 for converting torch FloatTensor
    y_np = np.cos(steps)

    x = Variable(torch.from_numpy(x_np[np.newaxis, :, np.newaxis]))    # shape (batch, time_step, input_size)
    y = Variable(torch.from_numpy(y_np[np.newaxis, :, np.newaxis]))

    prediction, h_state = rnn(x, h_state)   # rnn output
    # !! next step is important !!
    h_state = Variable(h_state.data)        # repack the hidden state, break the connection from last iteration

    loss = loss_func(prediction, y)         # calculate loss
    optimizer.zero_grad()                   # clear gradients for this training step
    loss.backward()                         # backpropagation, compute gradients
    optimizer.step()                        # apply gradients

    # plotting
    plt.plot(steps, y_np.flatten(), 'r-')
    plt.plot(steps, prediction.data.numpy().flatten(), 'b-')
    plt.draw(); plt.pause(0.05)

plt.ioff()
plt.show()

RNN(
  (rnn): RNN(1, 32, batch_first=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


In [658]:
cell=nn.RNN(batch_first=True,l)

SyntaxError: positional argument follows keyword argument (<ipython-input-658-4e7fac089080>, line 1)